In [1]:
# Imports
import os
import pandas as pd
import numpy as np
import datetime
import warnings
from scipy.optimize import minimize
from sklearn.metrics import r2_score
from IPython.display import display

# Definitions
pd.set_option('display.float_format', lambda x: '%.3f' % x)
%matplotlib inline
warnings.simplefilter('ignore')
N_JOBS = -1
SEED = 2017

In [2]:
# Stack all single Level 1 models OOF and test preds as entries for a Level 2 model
def get_level_one_outputs(OOF_targets) :
    level_one_OOF = pd.DataFrame()
    i = 0
    root_dir = "oof_preds/"
    for file_name in sorted(os.listdir(root_dir)) :
        preds_OOF = pd.read_csv(root_dir + file_name)
        cur_preds_OOF = pd.Series(preds_OOF.y, name = file_name[0:7])
        level_one_OOF = pd.concat([level_one_OOF, cur_preds_OOF], axis = 1)
        i += 1
    display(level_one_OOF.head())

    # Same for test set preds
    level_one_test = pd.DataFrame()
    i = 0
    root_dir = "preds/"
    for file_name in sorted(os.listdir(root_dir)) :
        preds_test = pd.read_csv(root_dir + file_name)
        cur_preds_test = pd.Series(preds_test.y, name = file_name[0:7])
        level_one_test = pd.concat([level_one_test, cur_preds_test], axis = 1)
        i += 1
    display(level_one_test.head())
    
    return(level_one_OOF, level_one_test)

In [3]:
'''# Define function to minimize
def target_func(weights) :
    final_prediction = 0
    for weight, prediction in zip(weights, preds_X_test):
        final_prediction += weight * prediction
    return(-r2_score(X_test_y, final_prediction))


# Optimize ensemble weights on held out fold
def optimize(preds_X_test, names) :
    scores = []
    weights = []
    for i in range(100):
        # Choose many random starting weights
        starting_values = np.random.uniform(size = len(preds_X_test))

        # Our weights are bound between 0 and 1
        bounds = [(0, 1)] * len(preds_X_test)

        res = minimize(
            target_func, 
            starting_values, 
            method = "SLSQP", 
            bounds = bounds, 
            options = {"maxiter" : 10000})

        scores.append(res["fun"])
        weights.append(res["x"])

    bestSC = -np.min(scores)
    bestWght = weights[np.argmin(scores)]

    print("\n Ensemble Score: {}".format(bestSC))
    print("\n Best Weights: {}".format(bestWght))
    print("\n Names: {}".format(names))

    return(bestSC, bestWght, names)'''

'# Define function to minimize\ndef target_func(weights) :\n    final_prediction = 0\n    for weight, prediction in zip(weights, preds_X_test):\n        final_prediction += weight * prediction\n    return(-r2_score(X_test_y, final_prediction))\n\n\n# Optimize ensemble weights on held out fold\ndef optimize(preds_X_test, names) :\n    scores = []\n    weights = []\n    for i in range(100):\n        # Choose many random starting weights\n        starting_values = np.random.uniform(size = len(preds_X_test))\n\n        # Our weights are bound between 0 and 1\n        bounds = [(0, 1)] * len(preds_X_test)\n\n        res = minimize(\n            target_func, \n            starting_values, \n            method = "SLSQP", \n            bounds = bounds, \n            options = {"maxiter" : 10000})\n\n        scores.append(res["fun"])\n        weights.append(res["x"])\n\n    bestSC = -np.min(scores)\n    bestWght = weights[np.argmin(scores)]\n\n    print("\n Ensemble Score: {}".format(bestSC))\n

In [9]:
# Define function to minimize
def target_func(weights) :
    final_prediction = 0
    for weight, prediction in zip(weights, preds_X_test):
        final_prediction += weight * prediction
    return(-r2_score(OOF_targets.y, final_prediction))


# Optimize ensemble weights on held out fold
def optimize(preds_X_test) :
    scores = []
    weights = []
    for i in range(100):
        # Choose many random starting weights
        starting_values = np.random.uniform(size = len(preds_X_test))

        # Our weights are bound between 0 and 1
        bounds = [(0, 1)] * len(preds_X_test)

        res = minimize(
            target_func, 
            starting_values, 
            method = "SLSQP", 
            bounds = bounds, 
            options = {"maxiter" : 10000})

        scores.append(res["fun"])
        weights.append(res["x"])

    bestSC = -np.min(scores)
    bestWght = weights[np.argmin(scores)]

    print("\n Ensemble Score: {}".format(bestSC))
    print("\n Best Weights: {}".format(bestWght))

    return(bestSC, bestWght)

<b>Script</b>

In [10]:
# Get data
OOF_targets = pd.read_csv("clean_data/OOF_targets.csv")
init_test = pd.read_csv("raw_data/test.csv")
test_ids = init_test.ID.values

display(OOF_targets.head())

,ID,y
0,1,99.150
1,2,91.980
2,3,91.520
3,4,91.570
4,5,90.110


In [11]:
# Create Level 2 inputs
level_one_OOF, level_one_test = get_level_one_outputs(OOF_targets)

,et_dc15,et_dc2_,et_dc3_,et_dc4_,et_dc5_,et_dc6_,et_dc7_
0,93.818,93.930,94.019,94.160,94.039,94.100,94.131
1,93.741,93.982,93.982,93.949,93.986,93.964,94.041
2,93.208,93.446,93.706,93.492,93.522,93.697,93.512
3,93.758,94.222,93.978,94.369,94.215,94.246,94.141
4,94.321,94.412,93.944,94.455,94.504,94.523,94.437


,et_dc15,et_dc2_,et_dc3_,et_dc4_,et_dc5_,et_dc6_,et_dc7_
0,78.668,78.365,77.753,78.544,78.549,79.078,78.543
1,93.689,93.904,93.871,93.892,93.962,94.017,93.963
2,77.303,77.510,77.420,77.199,77.466,77.166,77.539
3,78.276,78.111,77.434,78.561,78.164,78.466,78.226
4,112.993,112.932,112.707,113.328,112.907,113.103,113.004


In [14]:
# Get error of each OOF column
preds_X_test = []
for col in level_one_OOF.columns :
    cv_score = r2_score(OOF_targets.y, level_one_OOF[col])
    print("Global OOF r2_score for " + col + " : " + str(cv_score))

    preds_X_test.append(level_one_OOF[col])

Global OOF r2_score for et_dc15 : 0.565423151536
Global OOF r2_score for et_dc2_ : 0.564604887927
Global OOF r2_score for et_dc3_ : 0.566072957735
Global OOF r2_score for et_dc4_ : 0.563813793244
Global OOF r2_score for et_dc5_ : 0.564090667366
Global OOF r2_score for et_dc6_ : 0.56416472274
Global OOF r2_score for et_dc7_ : 0.564040091236


In [13]:
# Optimize weights
bestSC, bestWght = optimize(preds_X_test)


 Ensemble Score: 0.5671744331684818

 Best Weights: [  4.81152288e-01   7.16523087e-17   5.20450577e-01   2.39230851e-18
   4.48331193e-18   0.00000000e+00   0.00000000e+00]


In [36]:
# Prepare submission using optimal weights
preds = 0
for i in range(0, len(bestWght)) :
    preds = preds + (bestWght[i] * level_one_test[[i]].values)

In [48]:
pd.DataFrame(preds)[0]

0       78.318
1       93.934
2       77.488
3       77.963
4      113.025
5       93.491
6      112.674
7       93.892
8      116.375
9       94.237
10     115.576
11     110.244
12      97.439
13      94.232
14     103.920
15      99.346
16     117.134
17      98.485
18      94.867
19      95.807
20      98.485
21      98.485
22      94.553
23      95.980
24      93.894
25     118.407
26     102.807
27     103.584
28      94.089
29      77.425
         ...  
4179   111.291
4180   102.958
4181    93.298
4182    93.417
4183   102.759
4184   110.394
4185    92.955
4186    93.354
4187   111.953
4188   110.600
4189    93.121
4190   113.596
4191    92.696
4192   103.577
4193    93.181
4194   111.413
4195    93.395
4196   103.769
4197   102.370
4198   112.395
4199    92.956
4200    92.794
4201    93.271
4202   111.698
4203   110.803
4204   103.522
4205    95.201
4206    93.109
4207   110.576
4208    92.756
Name: 0, dtype: float64

In [49]:
# Save predictions
file_name = "ens_preds/avg_" + str(level_one_OOF.shape[1]) + ".csv"
pd.DataFrame({"ID": init_test.ID.values, "y": pd.DataFrame(preds)[0]}).to_csv(file_name, index = False)


In [ ]:
'''# Get validation set data
la_preds_X_test = np.array(pd.read_csv("preds/la_preds_X_test.csv", header = None))
et_preds_X_test = np.array(pd.read_csv("preds/et_preds_X_test.csv", header = None))
xg_preds_X_test = np.array(pd.read_csv("preds/xg_preds_X_test.csv", header = None))

# Get test set data
la_preds_test = np.array(pd.read_csv("preds/la_preds_test.csv"))
et_preds_test = np.array(pd.read_csv("preds/et_preds_test.csv"))
xg_preds_test = np.array(pd.read_csv("preds/xg_preds_test.csv"))

preds_X_test = []
preds_X_test.append(la_preds_X_test[:, 0])
preds_X_test.append(et_preds_X_test[:, 0])
preds_X_test.append(xg_preds_X_test[:, 0])

preds_test = []
preds_test.append(la_preds_test[:, 1])
preds_test.append(et_preds_test[:, 1])
preds_test.append(xg_preds_test[:, 1])

names = []
names.append("la")
names.append("et")
names.append("xg")'''


In [ ]:
'''# Scores of single models on validation set
print("la on held out fold : " + str(r2_score(X_test_y, la_preds_X_test)))
print("et on held out fold : " + str(r2_score(X_test_y, et_preds_X_test)))
print("xg on held out fold : " + str(r2_score(X_test_y, xg_preds_X_test)))'''


In [ ]:
'''# Optimize weights
bestSC, bestWght, names = optimize(preds_X_test, names)'''

In [ ]:
'''# Prepare submission using optimal weights
preds = 0
for i in range(0, len(bestWght)) :
    (bestWght[i] * preds_test[i])
    preds += (bestWght[i] * preds_test[i])'''

In [ ]:
'''# Save predictions
pd.DataFrame({"ID": test_ids[0], "y": preds}).to_csv("preds/blend.csv", index = False)'''
